In [1]:
# HMM smoother

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import os
from hmmlearn import hmm

from sklearn.ensemble import RandomForestClassifier
from sklearn import cross_validation
from sklearn.cross_validation import KFold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.grid_search import GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import PolynomialFeatures

from collections import deque
from itertools import islice

from wrangle_updated import trial
from utilities import convert_to_words, get_position_stats, combine_csv, resolve_acc_gyro, blank_filter, concat_data

TIME_SEQUENCE_LENGTH = 50
DIR = os.path.dirname(os.path.realpath('__file__'))
polynomial_features = PolynomialFeatures(interaction_only=False, include_bias=True, degree=3)

In [11]:
def root_sum_square(x, y, z):
        sum = ((x**2)+(y**2)+(z**2))
        rss = math.sqrt(sum)
        return rss

def root_mean_square(x, y, z):
        mean = ((x**2)+(y**2)+(z**2))/3
        rss = math.sqrt(mean)
        return rss

def tiltx(x, y, z):
    try:
        prep = (x/(math.sqrt((y**2)+(z**2))))
        tilt = math.atan(prep)
    except ZeroDivisionError:
        tilt = 0
    return tilt

def tilty(x, y, z):
    try:
        prep = (y/(math.sqrt((x**2)+(z**2))))
        tilt = math.atan(prep)
    except ZeroDivisionError:
        tilt = 0
    return tilt
    
def max_min_diff(max, min):
    diff = max - min
    return diff

def magnitude(x, y, z):
    magnitude = x + y + z
    return magnitude

def create_features(df, _window=50):
    accel_x = df['ACCEL_X'].astype(float)
    accel_y = df['ACCEL_Y'].astype(float)
    accel_z = df['ACCEL_Z'].astype(float)
    gyro_x = df['GYRO_X'].astype(float)
    gyro_y = df['GYRO_Y'].astype(float)
    gyro_z = df['GYRO_Z'].astype(float)
    
    df2 = pd.DataFrame()
    
    # Capture stand state here, then average later
    
    df2['stand'] = df['stand'].astype(float)
    
    TIME_SEQUENCE_LENGTH = _window
    
    # Basics
    
    df2['ACCEL_X'] = pd.rolling_mean(accel_x, TIME_SEQUENCE_LENGTH-2, center=True)
    df2['ACCEL_Y'] = pd.rolling_mean(accel_y, TIME_SEQUENCE_LENGTH-2, center=True)
    df2['ACCEL_Z'] = pd.rolling_mean(accel_z, TIME_SEQUENCE_LENGTH-2, center=True)
    df2['GYRO_X'] = pd.rolling_mean(gyro_x, TIME_SEQUENCE_LENGTH-2, center=True)
    df2['GYRO_Y'] = pd.rolling_mean(gyro_y, TIME_SEQUENCE_LENGTH-2, center=True)
    df2['GYRO_Z'] = pd.rolling_mean(gyro_z, TIME_SEQUENCE_LENGTH-2, center=True)
    
    # standing up detection
    df2['avg_stand'] = pd.rolling_mean(df2['stand'], TIME_SEQUENCE_LENGTH-2, center=True)
    print df2['avg_stand']
    
    # round standing up as we need it to be either '0' or '1' for training later
    df2['avg_stand'] = df2['avg_stand'].apply(lambda x: math.ceil(x))
    
    print df2['avg_stand']

    ol_upper = _window/2
    ol_lower = ol_upper-1
        
    new_df = df2[ol_lower::ol_upper] # 50% overlap with 30

    return new_df


# Test method:
# data = np.array([np.mean(training_data.ACCEL_X[0:30]), np.mean(training_data.ACCEL_X[30:45]), np.mean(training_data.ACCEL_X[30:60])])
# desired_df = pd.DataFrame(data, columns=columns)
# print desired_df


In [12]:
def set_state(df, state):
    """set the state for training"""

    if state == 'your_mount':
        df['state'] = 0
    elif state == 'your_side_control':
        df['state'] = 1
    elif state =='your_closed_guard':
        df['state'] = 2
    elif state =='your_back_control':
        df['state'] = 3
    elif state =='opponent_mount_or_sc':
        df['state'] = 4
    elif state =='opponent_closed_guard':
        df['state'] = 5
    elif state == 'opponent_back_control':
        df['state'] = 6
    elif state =='non_jj':
        df['state'] = 7
        
    return df

In [13]:
def set_stand_state(df, stand_state):
    if (stand_state == 1):
        df['stand'] = 1
    else:
        df['stand'] = 0
    
    print df
    return df

In [14]:
def combine_setState_createFeatures(directory, state, window=50, stand=0):
    """
    convenience method to combine three steps in one function:
    (1) combine multiple csv files, (2) set their movement state for training,
    (3) combine to create time sequences and add features
    """
    combined_data = combine_csv(directory)
    combined_data_updated = set_state(combined_data, state)
    combined_data_updated2 = set_stand_state(combined_data_updated, stand)
    feature_training_data = create_features(combined_data_updated2, window)
    ready_training_data = set_state(feature_training_data, state)
    return ready_training_data

In [15]:
def prep(window=30):
    """prepare the raw sensor data"""

    #1 Your mount
    ymount_td = combine_setState_createFeatures('your_mount_raw_data', 'your_mount', window, 0)
    #2 Your side control
    ysc_td = combine_setState_createFeatures('your_side_control_raw_data', 'your_side_control', window, 0)
    #3 Your closed guard
    ycg_td = combine_setState_createFeatures('your_closed_guard_raw_data', 'your_closed_guard', window, 0)
    #4 Your back control
    ybc_td = combine_setState_createFeatures('your_back_control_raw_data', 'your_back_control', window, 0)
    #5 Opponent mount or opponent side control
    omountsc_td = combine_setState_createFeatures('opponent_mount_and_opponent_side_control_raw_data', 'opponent_mount_or_sc', window, 0)
    #6 Opponent closed guard
    ocg_td = combine_setState_createFeatures('opponent_closed_guard_raw_data', 'opponent_closed_guard', window, 0)
    #7 Opponent back control
    obc_td = combine_setState_createFeatures('opponent_back_control_raw_data', 'opponent_back_control', window, 0)
    #8 "Non jiu-jitsu" motion
    nonjj_td = combine_setState_createFeatures('non_jj_raw_data', 'non_jj', window, 0)
    #9 "stand up" motion
    stand_up_td = combine_setState_createFeatures('standing_up_raw_data', 'opponent_closed_guard', window, 1)

    training_data = concat_data([ymount_td, ysc_td, ycg_td, ybc_td, omountsc_td, ocg_td, obc_td, nonjj_td])
    # remove NaN
    training_data = blank_filter(training_data)
    return training_data

In [16]:
def prep_test(el_file):
    el_file = 'data/test_cases/' + el_file
    df = pd.DataFrame()
    df = pd.read_csv(el_file, index_col=None, header=0)
    df = resolve_acc_gyro(df)
    df = create_features(df)
    test_data = blank_filter(df)

    return test_data

In [17]:
def test_model_stand(df_train):
    """check model accuracy"""

    y = df_train['stand'].values
    X = df_train.drop(['stand', 'state', 'index'], axis=1)
    
    if X.isnull().values.any() == False: 

        rf = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                max_depth=None, max_features='auto', max_leaf_nodes=None,
                min_samples_leaf=8, min_samples_split=4,
                min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=-1,
                oob_score=False, random_state=None, verbose=0,
                warm_start=False)
        
        X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, y, test_size=0.1)
        
    rf.fit(X_train, y_train)
    rf_pred = rf.predict(X_test)
    rf_scores = cross_validation.cross_val_score(
    rf, X, df_train.state, cv=10, scoring='accuracy')
    
    print 'rf prediction: {}'.format(accuracy_score(y_test, rf_pred))
    print("Random Forest Accuracy: %0.2f (+/- %0.2f)" % (rf_scores.mean(), rf_scores.std() * 2))
    
    importances = rf.feature_importances_
    std = np.std([tree.feature_importances_ for tree in rf.estimators_],
             axis=0)
    indices = np.argsort(importances)[::-1]

    # Print the feature ranking
    print("Feature ranking:")
    for f in range(X.shape[1]):
        print("%d. feature %s (%f)" % (f + 1, X.columns[indices[f]], importances[indices[f]]))
    

In [18]:
def test_model(df_train):
    """check model accuracy"""

    y = df_train['state'].values
    X = df_train.drop(['state', 'index'], axis=1)
    
    if X.isnull().values.any() == False: 

        rf = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                max_depth=None, max_features='auto', max_leaf_nodes=None,
                min_samples_leaf=8, min_samples_split=4,
                min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=-1,
                oob_score=False, random_state=None, verbose=0,
                warm_start=False)
        
        X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, y, test_size=0.1)
        
    rf.fit(X_train, y_train)
    rf_pred = rf.predict(X_test)
    rf_scores = cross_validation.cross_val_score(
    rf, X, df_train.state, cv=10, scoring='accuracy')
    
    print 'rf prediction: {}'.format(accuracy_score(y_test, rf_pred))
    print("Random Forest Accuracy: %0.2f (+/- %0.2f)" % (rf_scores.mean(), rf_scores.std() * 2))
    
    importances = rf.feature_importances_
    std = np.std([tree.feature_importances_ for tree in rf.estimators_],
             axis=0)
    indices = np.argsort(importances)[::-1]

    # Print the feature ranking
    print("Feature ranking:")
    for f in range(X.shape[1]):
        print("%d. feature %s (%f)" % (f + 1, X.columns[indices[f]], importances[indices[f]]))

In [19]:
training_data50 = prep(50)

['/Users/christophersamiullah/repos/sensor_readings/ML_Sandbox/data/your_mount_raw_data/DIO_YMOUNT.csv', '/Users/christophersamiullah/repos/sensor_readings/ML_Sandbox/data/your_mount_raw_data/GL_ymount_CS.csv', '/Users/christophersamiullah/repos/sensor_readings/ML_Sandbox/data/your_mount_raw_data/GL_ymount_UrsWearing.csv', '/Users/christophersamiullah/repos/sensor_readings/ML_Sandbox/data/your_mount_raw_data/ymount.csv', '/Users/christophersamiullah/repos/sensor_readings/ML_Sandbox/data/your_mount_raw_data/ymountUrs.csv', '/Users/christophersamiullah/repos/sensor_readings/ML_Sandbox/data/your_mount_raw_data/ymountUrs2.csv']
       index             timestamp_x  ACCEL_X  ACCEL_Y  ACCEL_Z             timestamp_y   GYRO_X   GYRO_Y   GYRO_Z  state  stand
0          0  2016-03-26 13:33:12.67   -0.358    0.636    0.721  2016-03-26 13:33:12.00   21.829 -123.049   -2.622      0      0
1          1  2016-03-26 13:33:12.67   -0.071   -0.115   -1.280  2016-03-26 13:33:12.00   27.012  -63.902  -14

In [21]:
print training_data50.avg_stand.describe()

count    5367
mean        0
std         0
min         0
25%         0
50%         0
75%         0
max         0
Name: avg_stand, dtype: float64


In [200]:
training_data = prep(30)
training_data10 = prep(10)
training_data20 = prep(20)
training_data26 = prep(26) # need numbers that divide into 2 easily
training_data36 = prep(36) # need numbers that divide into 2 easily
training_data40 = prep(40)
#training_data50 = prep(50)
training_data56 = prep(56)
training_data60 = prep(60)
training_data64 = prep(64)
training_data70 = prep(70)


['/Users/christophersamiullah/repos/sensor_readings/ML_Sandbox/data/your_mount_raw_data/GL_ymount_CS.csv', '/Users/christophersamiullah/repos/sensor_readings/ML_Sandbox/data/your_mount_raw_data/GL_ymount_UrsWearing.csv', '/Users/christophersamiullah/repos/sensor_readings/ML_Sandbox/data/your_mount_raw_data/ymount.csv', '/Users/christophersamiullah/repos/sensor_readings/ML_Sandbox/data/your_mount_raw_data/ymountUrs.csv', '/Users/christophersamiullah/repos/sensor_readings/ML_Sandbox/data/your_mount_raw_data/ymountUrs2.csv']


/Users/christophersamiullah/repos/sensor_readings/venv/lib/python2.7/site-packages/ipykernel/__main__.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/christophersamiullah/repos/sensor_readings/venv/lib/python2.7/site-packages/ipykernel/__main__.py:86: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/christophersamiullah/repos/sensor_readings/venv/lib/python2.7/site-packages/ipykernel/__main__.py:88: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_

['/Users/christophersamiullah/repos/sensor_readings/ML_Sandbox/data/your_side_control_raw_data/GL_ysc_UrsWearing.csv', '/Users/christophersamiullah/repos/sensor_readings/ML_Sandbox/data/your_side_control_raw_data/ysc1.csv', '/Users/christophersamiullah/repos/sensor_readings/ML_Sandbox/data/your_side_control_raw_data/ysc2.csv', '/Users/christophersamiullah/repos/sensor_readings/ML_Sandbox/data/your_side_control_raw_data/yscUrs.csv']
['/Users/christophersamiullah/repos/sensor_readings/ML_Sandbox/data/your_closed_guard_raw_data/GL_ycg_UrsWearing.csv', '/Users/christophersamiullah/repos/sensor_readings/ML_Sandbox/data/your_closed_guard_raw_data/ycg.csv', '/Users/christophersamiullah/repos/sensor_readings/ML_Sandbox/data/your_closed_guard_raw_data/ycg2Urs.csv', '/Users/christophersamiullah/repos/sensor_readings/ML_Sandbox/data/your_closed_guard_raw_data/ycgUrs.csv']
['/Users/christophersamiullah/repos/sensor_readings/ML_Sandbox/data/your_back_control_raw_data/GL_ybc_UrsWearing.csv', '/Users

In [43]:
print training_data50[training_data50.avg_stand == 0]

      index     tiltx     tilty  stand   ACCEL_X   ACCEL_Y   ACCEL_Z     GYRO_X     GYRO_Y     GYRO_Z  ...    max_min_gx  max_min_gy  max_min_gz   acc_rss   gyro_rss   acc_rms   gyro_rms  acc_magnitude  gyro_magnitude  state
0        24 -0.337718 -0.074182      0 -0.298854  0.061854 -0.905396  -3.197437  -8.581000   2.266271  ...       287.927     287.927     287.927  0.955448   9.433618  0.551628   5.446502      -1.142396       -9.512167      0
1        49  0.078648  0.178678      0 -0.249208  0.099646 -0.944208   8.991375  -3.570812   2.945875  ...       279.695     279.695     279.695  0.981613  10.113046  0.566734   5.838770      -1.093771        8.366438      0
2        74 -0.308385  0.009462      0 -0.186458  0.086958 -0.948500 -10.496646  -1.448104   0.388729  ...       295.853     295.853     295.853  0.970557  10.603192  0.560351   6.121756      -1.048000      -11.556021      0
3        99  0.442236  0.669933      0 -0.096833  0.180833 -0.947500  -4.448667   6.206813  14.32289

In [177]:
#1 Your mount
ymount_td = combine_setState_createFeatures('your_mount_raw_data', 'your_mount')
#2 Your side control
ysc_td = combine_setState_createFeatures('your_side_control_raw_data', 'your_side_control')
#3 Your closed guard
ycg_td = combine_setState_createFeatures('your_closed_guard_raw_data', 'your_closed_guard')
#4 Your back control
ybc_td = combine_setState_createFeatures('your_back_control_raw_data', 'your_back_control')
#5 Opponent mount or opponent side control
omountsc_td = combine_setState_createFeatures('opponent_mount_and_opponent_side_control_raw_data', 'opponent_mount_or_sc')
#6 Opponent closed guard
ocg_td = combine_setState_createFeatures('opponent_closed_guard_raw_data', 'opponent_closed_guard')
#7 Opponent back control
obc_td = combine_setState_createFeatures('opponent_back_control_raw_data', 'opponent_back_control')
#8 "Non jiu-jitsu" motion
nonjj_td = combine_setState_createFeatures('non_jj_raw_data', 'non_jj')

['/Users/christophersamiullah/repos/sensor_readings/ML_Sandbox/data/your_mount_raw_data/GL_ymount_CS.csv', '/Users/christophersamiullah/repos/sensor_readings/ML_Sandbox/data/your_mount_raw_data/GL_ymount_UrsWearing.csv', '/Users/christophersamiullah/repos/sensor_readings/ML_Sandbox/data/your_mount_raw_data/ymount.csv', '/Users/christophersamiullah/repos/sensor_readings/ML_Sandbox/data/your_mount_raw_data/ymountUrs.csv', '/Users/christophersamiullah/repos/sensor_readings/ML_Sandbox/data/your_mount_raw_data/ymountUrs2.csv']


/Users/christophersamiullah/repos/sensor_readings/venv/lib/python2.7/site-packages/ipykernel/__main__.py:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/christophersamiullah/repos/sensor_readings/venv/lib/python2.7/site-packages/ipykernel/__main__.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/christophersamiullah/repos/sensor_readings/venv/lib/python2.7/site-packages/ipykernel/__main__.py:82: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_

['/Users/christophersamiullah/repos/sensor_readings/ML_Sandbox/data/your_side_control_raw_data/GL_ysc_UrsWearing.csv', '/Users/christophersamiullah/repos/sensor_readings/ML_Sandbox/data/your_side_control_raw_data/ysc1.csv', '/Users/christophersamiullah/repos/sensor_readings/ML_Sandbox/data/your_side_control_raw_data/ysc2.csv', '/Users/christophersamiullah/repos/sensor_readings/ML_Sandbox/data/your_side_control_raw_data/yscUrs.csv']
['/Users/christophersamiullah/repos/sensor_readings/ML_Sandbox/data/your_closed_guard_raw_data/GL_ycg_UrsWearing.csv', '/Users/christophersamiullah/repos/sensor_readings/ML_Sandbox/data/your_closed_guard_raw_data/ycg.csv', '/Users/christophersamiullah/repos/sensor_readings/ML_Sandbox/data/your_closed_guard_raw_data/ycg2Urs.csv', '/Users/christophersamiullah/repos/sensor_readings/ML_Sandbox/data/your_closed_guard_raw_data/ycgUrs.csv']
['/Users/christophersamiullah/repos/sensor_readings/ML_Sandbox/data/your_back_control_raw_data/GL_ybc_UrsWearing.csv', '/Users

In [166]:
test_data1 = prep_test('test1_ymount_ycg.csv')
test_data4 = prep_test('GL_TEST1_CS.csv')
test_data5 = prep_test('GL_TEST2_CS.csv')
test_data6 = prep_test('GL_TEST3_CS_very_still.csv')
test_data7 = prep_test('GL_TEST1_UrsWearing.csv')

test_data100 = prep_test('CS_OCG_STAND_OCG.csv')


/Users/christophersamiullah/repos/sensor_readings/venv/lib/python2.7/site-packages/ipykernel/__main__.py:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/christophersamiullah/repos/sensor_readings/venv/lib/python2.7/site-packages/ipykernel/__main__.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/christophersamiullah/repos/sensor_readings/venv/lib/python2.7/site-packages/ipykernel/__main__.py:82: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_

Removed 2 NaN rows
Removed 2 NaN rows
Removed 1 NaN rows
Removed 5 NaN rows
Removed 1 NaN rows
   index   ACCEL_X   ACCEL_Y   ACCEL_Z    GYRO_X    GYRO_Y  GYRO_Z  rolling_median_x  rolling_median_y  rolling_median_z  ...    rolling_std_y  rolling_std_z  rolling_std_gx  rolling_std_gy  rolling_std_gz   acc_rss  gyro_rss  acc_rms  gyro_rms  state
0     14 -0.957107 -0.142429  0.343357  1.631143 -3.353714   2.633           -0.9605            -0.147             0.341  ...         0.023024       0.028165        3.959507        3.959507        3.959507  1.026759  4.565163   0.5928  2.635698      0

[1 rows x 36 columns]
     ACCEL_X   ACCEL_Y   ACCEL_Z  GYRO_X    GYRO_Y    GYRO_Z  rolling_median_x  rolling_median_y  rolling_median_z  rolling_median_gx    ...     rolling_std_x  rolling_std_y  rolling_std_z  rolling_std_gx  rolling_std_gy  rolling_std_gz   acc_rss  gyro_rss   acc_rms  gyro_rms
14 -0.973643 -0.279821  0.168107    0.44  0.568429  0.723107           -0.9755           -0.2795     

In [15]:
print training_data50.columns

Index([u'index', u'tiltx', u'tilty', u'ACCEL_X', u'ACCEL_Y', u'ACCEL_Z', u'GYRO_X', u'GYRO_Y', u'GYRO_Z', u'rolling_median_x', u'rolling_median_y', u'rolling_median_z', u'rolling_median_gx', u'rolling_median_gy', u'rolling_median_gz', u'rolling_max_x', u'rolling_max_y', u'rolling_max_z', u'rolling_max_gx', u'rolling_max_gy', u'rolling_max_gz', u'rolling_min_x', u'rolling_min_y', u'rolling_min_z', u'rolling_min_gx', u'rolling_min_gy', u'rolling_min_gz', u'rolling_sum_x', u'rolling_sum_y', u'rolling_sum_z', u'rolling_sum_gx', u'rolling_sum_gy', u'rolling_sum_gz', u'rolling_std_x', u'rolling_std_y', u'rolling_std_z', u'rolling_std_gx', u'rolling_std_gy', u'rolling_std_gz', u'avg_tiltx', u'avg_tilty', u'max_min_x', u'max_min_y', u'max_min_z', u'max_min_gx', u'max_min_gy', u'max_min_gz', u'acc_rss', u'gyro_rss', u'acc_rms', u'gyro_rms', u'acc_magnitude', u'gyro_magnitude', u'state'], dtype='object')


In [155]:
pre_smooth = trial(training_data, test_data1)

[7 7 7 7 0 5 5 5 5 0 0 5 5 5 5 5 0 0 5 5 5 0 5 7 7 5 5 5 5 7 7 5 5 0 1 1 6
 3 4 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 4 4 2 2 2 2 2 2 4 2 2 2 2 2 2 4 4 4 4
 4 4 4 4 4 4 4 4]
['OTHER', 'OTHER', 'OTHER', 'OTHER', 'your_mount', 'opponent_closed_guard', 'opponent_closed_guard', 'opponent_closed_guard', 'opponent_closed_guard', 'your_mount', 'your_mount', 'opponent_closed_guard', 'opponent_closed_guard', 'opponent_closed_guard', 'opponent_closed_guard', 'opponent_closed_guard', 'your_mount', 'your_mount', 'opponent_closed_guard', 'opponent_closed_guard', 'opponent_closed_guard', 'your_mount', 'opponent_closed_guard', 'OTHER', 'OTHER', 'opponent_closed_guard', 'opponent_closed_guard', 'opponent_closed_guard', 'opponent_closed_guard', 'OTHER', 'OTHER', 'opponent_closed_guard', 'opponent_closed_guard', 'your_mount', 'your_side_control', 'your_side_control', 'opponent_back_control', 'your_back_control', 'opponent_mount_or_sc', 'your_closed_guard', 'your_closed_guard', 'your_closed_guard', 'your_close

In [12]:
test_model_stand(training_data50) # newest with tilt

KeyError: 'stand'

In [204]:
print '*** 30 rows ***'
test_model(training_data)

print '*** 10 rows ***'
test_model(training_data10)

print '*** 20 rows ***'
test_model(training_data20)

print '*** 26 rows ***'
test_model(training_data26)

print '*** 36 rows ***'
test_model(training_data36)

print '*** 40 rows ***'
test_model(training_data40)

print '*** 50 rows ***'
test_model(training_data50)

print '*** 60 rows ***'
test_model(training_data60)

print '*** 56 rows ***'
test_model(training_data56)

print '*** 64 rows ***'
test_model(training_data64)

print '*** 70 rows ***'
test_model(training_data70)

*** 30 rows ***
rf prediction: 0.810615199035
Random Forest Accuracy: 0.73 (+/- 0.15)
Feature ranking:
1. feature rolling_max_z (0.100646)
2. feature ACCEL_Z (0.087646)
3. feature rolling_median_z (0.077526)
4. feature rolling_min_z (0.067189)
5. feature ACCEL_X (0.064614)
6. feature rolling_max_x (0.060364)
7. feature rolling_median_x (0.055946)
8. feature rolling_min_x (0.051143)
9. feature acc_rms (0.040409)
10. feature acc_rss (0.040108)
11. feature rolling_std_x (0.033889)
12. feature ACCEL_Y (0.029493)
13. feature rolling_median_y (0.028272)
14. feature rolling_std_y (0.026964)
15. feature rolling_max_y (0.024981)
16. feature rolling_min_y (0.024974)
17. feature rolling_std_gz (0.020580)
18. feature rolling_std_gy (0.019857)
19. feature rolling_std_gx (0.019550)
20. feature rolling_std_z (0.018499)
21. feature rolling_min_gx (0.009949)
22. feature rolling_max_gx (0.009949)
23. feature rolling_max_gy (0.009839)
24. feature gyro_rss (0.009706)
25. feature rolling_max_gz (0.009641)


In [205]:
print training_data70

      index   ACCEL_X   ACCEL_Y   ACCEL_Z    GYRO_X     GYRO_Y    GYRO_Z  rolling_median_x  rolling_median_y  rolling_median_z  ...    rolling_std_y  rolling_std_z  rolling_std_gx  rolling_std_gy  rolling_std_gz   acc_rss   gyro_rss   acc_rms   gyro_rms  state
0        34 -0.960191 -0.159721  0.313868 -2.416632   0.650971 -1.270588           -0.9620           -0.1450            0.3145  ...         0.060615       0.053717       13.342895       13.342895       13.342895  1.022737   2.806825  0.590477   1.620521      0
1        69 -0.982074 -0.210000  0.102059 -5.165956   7.133265  0.401721           -0.9820           -0.1740            0.0570  ...         0.082127       0.179222       19.247336       19.247336       19.247336  1.009448   8.816572  0.582805   5.090250      0
2       104 -1.008515 -0.184912 -0.044074 -0.022397   4.052206  4.580324           -1.0115           -0.1565           -0.0300  ...         0.067879       0.045111       14.121794       14.121794       14.121794  1.02

In [180]:
pre_smooth2 = trial(training_data, test_data4)

[7 7 7 3 3 3 6 3 3 3 3 3 3 3 3 3 3 7 5 0 0 0 0 0 0 0 0 0 0 0 0 5 0 0 1 1 1
 0 1 1 1 6 6 1 1 1 1 1 1 1 1 1 0 0 7 4 4 4 4 4 4 4 4 2 4 2 2 2 4 4 4 4 2 2
 3 2 2 2 2 2 2 2 4 4 2 7 0 5 0 0 0 5 5 5 5 5 5 5 1 5 0 5 5 0 0 5 5 7 2 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 2 6 6 6 6 4 4 6 6 6 6 6 6 6
 6 6 6 6 6 6 6 6 6 6 1 0 0 7]
['OTHER', 'OTHER', 'OTHER', 'your_back_control', 'your_back_control', 'your_back_control', 'opponent_back_control', 'your_back_control', 'your_back_control', 'your_back_control', 'your_back_control', 'your_back_control', 'your_back_control', 'your_back_control', 'your_back_control', 'your_back_control', 'your_back_control', 'OTHER', 'opponent_closed_guard', 'your_mount', 'your_mount', 'your_mount', 'your_mount', 'your_mount', 'your_mount', 'your_mount', 'your_mount', 'your_mount', 'your_mount', 'your_mount', 'your_mount', 'opponent_closed_guard', 'your_mount', 'your_mount', 'your_side_control', 'your_side_control', 'your_side_control', 'your_mount', 'your_side_c

In [9]:
#pre_smooth3 = trial(training_data, test_data5)

[7 7 6 1 6 6 1 6 6 6 3 3 3 1 0 5 0 0 0 0 1 1 1 1 1 1 1 0 4 2 2 2 2 2 2 2 2
 2 2 2 7 0 5 5 0 5 5 5 5 5 5 5 5 7 4 4 4 4 2 4 2 4 4 4 2 4 4 4 4 2 4 3 6 2
 4 6 3 4 6 6 6 6 6 6 6 6 6 7 5 7]
['OTHER', 'OTHER', 'opponent_back_control', 'your_side_control', 'opponent_back_control', 'opponent_back_control', 'your_side_control', 'opponent_back_control', 'opponent_back_control', 'opponent_back_control', 'your_back_control', 'your_back_control', 'your_back_control', 'your_side_control', 'your_mount', 'opponent_closed_guard', 'your_mount', 'your_mount', 'your_mount', 'your_mount', 'your_side_control', 'your_side_control', 'your_side_control', 'your_side_control', 'your_side_control', 'your_side_control', 'your_side_control', 'your_mount', 'opponent_mount_or_sc', 'your_closed_guard', 'your_closed_guard', 'your_closed_guard', 'your_closed_guard', 'your_closed_guard', 'your_closed_guard', 'your_closed_guard', 'your_closed_guard', 'your_closed_guard', 'your_closed_guard', 'your_closed_guard', 'OTHER', '

In [10]:
#pre_smooth4 = trial(training_data, test_data6)

[7 7 7 6 3 3 6 6 6 3 3 3 5 0 0 5 5 5 5 5 5 7 7 7 7 5 7 5 0 1 1 1 1 1 1 1 1
 1 1 1 5 7 2 2 2 2 7 7 7 2 2 2 7 7 7 2 2 7 7 7 7 2 2 2 5 5 5 5 5 5 5 5 5 5
 5 5 5 5 5 0 5 7 4 4 4 4 4 7 7 7 7 7 7 7 7 7 2 4 4 4 4 4 4 4 4 7 4 7 7 7 7
 4 4 2 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 0 0 7 5]
['OTHER', 'OTHER', 'OTHER', 'opponent_back_control', 'your_back_control', 'your_back_control', 'opponent_back_control', 'opponent_back_control', 'opponent_back_control', 'your_back_control', 'your_back_control', 'your_back_control', 'opponent_closed_guard', 'your_mount', 'your_mount', 'opponent_closed_guard', 'opponent_closed_guard', 'opponent_closed_guard', 'opponent_closed_guard', 'opponent_closed_guard', 'opponent_closed_guard', 'OTHER', 'OTHER', 'OTHER', 'OTHER', 'opponent_closed_guard', 'OTHER', 'opponent_closed_guard', 'your_mount', 'your_side_control', 'your_side_control', 'your_side_control', 'your_side_control', 'your_side_control', 'your_side_control', 'your_side_control', 'your_side_control', 'your_side_c

In [11]:
#pre_smooth5 = trial(training_data, test_data7)

[7 7 2 3 3 3 3 3 3 3 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 0 4 4 4 4 4 4 2 2 4 4
 1 0 0 0 3 0 0 0 5 0 4 4 4 4 4 4 4 4 4 4 4 4 2 4 4 4 6 3 6 6 6 6 6 6 6 6 6
 1 1 5 7]
['OTHER', 'OTHER', 'your_closed_guard', 'your_back_control', 'your_back_control', 'your_back_control', 'your_back_control', 'your_back_control', 'your_back_control', 'your_back_control', 'your_mount', 'your_mount', 'your_mount', 'your_mount', 'your_mount', 'your_mount', 'your_mount', 'your_mount', 'your_mount', 'your_side_control', 'your_side_control', 'your_side_control', 'your_side_control', 'your_side_control', 'your_side_control', 'your_side_control', 'your_mount', 'opponent_mount_or_sc', 'opponent_mount_or_sc', 'opponent_mount_or_sc', 'opponent_mount_or_sc', 'opponent_mount_or_sc', 'opponent_mount_or_sc', 'your_closed_guard', 'your_closed_guard', 'opponent_mount_or_sc', 'opponent_mount_or_sc', 'your_side_control', 'your_mount', 'your_mount', 'your_mount', 'your_back_control', 'your_mount', 'your_mount', 'your_mount', 'oppon

In [110]:
print pre_smooth
pre_smooth_words = convert_to_words(pre_smooth)
pre_smooth_words2 = convert_to_words(pre_smooth2)
#pre_smooth_words3 = convert_to_words(pre_smooth3)
#pre_smooth_words4 = convert_to_words(pre_smooth4)
#pre_smooth_words5 = convert_to_words(pre_smooth5)
print pre_smooth_words

[7 7 7 0 0 0 5 5 0 5 0 5 5 0 5 7 5 0 6 4 2 2 2 2 2 2 2 2 4 2 2 2 2 2 2 4 4
 4 4 4 4]
['OTHER', 'OTHER', 'OTHER', 'your_mount', 'your_mount', 'your_mount', 'opponent_closed_guard', 'opponent_closed_guard', 'your_mount', 'opponent_closed_guard', 'your_mount', 'opponent_closed_guard', 'opponent_closed_guard', 'your_mount', 'opponent_closed_guard', 'OTHER', 'opponent_closed_guard', 'your_mount', 'opponent_back_control', 'opponent_mount_or_sc', 'your_closed_guard', 'your_closed_guard', 'your_closed_guard', 'your_closed_guard', 'your_closed_guard', 'your_closed_guard', 'your_closed_guard', 'your_closed_guard', 'opponent_mount_or_sc', 'your_closed_guard', 'your_closed_guard', 'your_closed_guard', 'your_closed_guard', 'your_closed_guard', 'your_closed_guard', 'opponent_mount_or_sc', 'opponent_mount_or_sc', 'opponent_mount_or_sc', 'opponent_mount_or_sc', 'opponent_mount_or_sc', 'opponent_mount_or_sc']


In [111]:
n_components = 8 # ('ybc', 'ymount', 'ysc', 'ycg', 'ocg', 'osc_mount', 'obc', 'other')
# n_components = 3
startprob = np.array([0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.65,]) # users will probably turn on sensor standing
# startprob = np.array([0.34, 0.33, 0.33])

In [112]:
# transmat = np.array([[0.34, 0.33, 0.33], [0.9, 0.05, 0.05], [0.9, 0.05, 0.05]])

"""
probability of these positions given current state:

your_mount' if v == 0 
else 'your_side_control' if v == 1
else 'your_closed_guard' if v == 2
else 'your_back_control' if v == 3
else 'opponent_mount_or_sc' if v == 4
else 'opponent_closed_guard' if v == 5
else 'opponent_back_control' if v == 6
else 'OTHER' if v == 7

transition_probability = {
        'ymt' : {'ymount': 0.800, 'ysc': 0.050, 'ycg': 0.010, 'ybc': 0.050, 'osc_mount': 0.001, 'ocg': 0.050, 'obc': 0.001, 'other': 0.038},
        'ysc' : {'ymount': 0.100, 'ysc': 0.800, 'ycg': 0.010, 'ybc': 0.010, 'osc_mount': 0.001, 'ocg': 0.050, 'obc': 0.001, 'other': 0.028},
        'ycg' : {'ymount': 0.010, 'ysc': 0.050, 'ycg': 0.800, 'ybc': 0.010, 'osc_mount': 0.050, 'ocg': 0.001, 'obc': 0.001, 'other': 0.078},
        'ybc' : {'ymount': 0.050, 'ysc': 0.010, 'ycg': 0.050, 'ybc': 0.800, 'osc_mount': 0.001, 'ocg': 0.010, 'obc': 0.001, 'other': 0.078},
        'omt' : {'ymount': 0.001, 'ysc': 0.050, 'ycg': 0.010, 'ybc': 0.001, 'osc_mount': 0.800, 'ocg': 0.050, 'obc': 0.050, 'other': 0.038},
        'ocg' : {'ymount': 0.100, 'ysc': 0.050, 'ycg': 0.010, 'ybc': 0.010, 'osc_mount': 0.001, 'ocg': 0.800, 'obc': 0.001, 'other': 0.028},
        'obc' : {'ymount': 0.010, 'ysc': 0.050, 'ycg': 0.001, 'ybc': 0.010, 'osc_mount': 0.050, 'ocg': 0.001, 'obc': 0.800, 'other': 0.078},
        'oth' : {'ymount': 0.050, 'ysc': 0.010, 'ycg': 0.050, 'ybc': 0.078, 'osc_mount': 0.001, 'ocg': 0.010, 'obc': 0.001, 'other': 0.800}
     }
"""

transmat = np.array([
                    [0.800, 0.050, 0.010, 0.050, 0.001, 0.050, 0.001, 0.038], 
                    [0.100, 0.800, 0.010, 0.010, 0.001, 0.050, 0.001, 0.028], 
                    [0.010, 0.050, 0.800, 0.010, 0.050, 0.001, 0.001, 0.078], 
                    [0.050, 0.010, 0.050, 0.800, 0.001, 0.010, 0.001, 0.078],
                    [0.001, 0.050, 0.010, 0.001, 0.800, 0.050, 0.050, 0.038],
                    [0.100, 0.050, 0.010, 0.010, 0.001, 0.800, 0.001, 0.028],
                    [0.010, 0.050, 0.001, 0.010, 0.050, 0.001, 0.800, 0.078],
                    [0.050, 0.010, 0.050, 0.078, 0.001, 0.010, 0.001, 0.800],
                    ])

In [113]:
# emissionprob = np.array([[0.34, 0.33, 0.33], [0.4, 0.55, 0.05], [0.05, 0.55, 0.4]])

"""

probability of these positions given current state:

your_mount' if v == 0 
else 'your_side_control' if v == 1
else 'your_closed_guard' if v == 2
else 'your_back_control' if v == 3
else 'opponent_mount_or_sc' if v == 4
else 'opponent_closed_guard' if v == 5
else 'opponent_back_control' if v == 6
else 'OTHER' if v == 7

emission_probability = {
        'ymt' : {'ymount': 0.500, 'ysc': 0.050, 'ycg': 0.010, 'ybc': 0.050, 'osc_mount': 0.001, 'ocg': 0.350, 'obc': 0.001, 'other': 0.038},
        'ysc' : {'ymount': 0.100, 'ysc': 0.800, 'ycg': 0.010, 'ybc': 0.010, 'osc_mount': 0.001, 'ocg': 0.050, 'obc': 0.001, 'other': 0.028},
        'ycg' : {'ymount': 0.010, 'ysc': 0.050, 'ycg': 0.400, 'ybc': 0.010, 'osc_mount': 0.500, 'ocg': 0.001, 'obc': 0.001, 'other': 0.078},
        'ybc' : {'ymount': 0.050, 'ysc': 0.010, 'ycg': 0.050, 'ybc': 0.600, 'osc_mount': 0.001, 'ocg': 0.010, 'obc': 0.201, 'other': 0.078},
        'omt' : {'ymount': 0.001, 'ysc': 0.050, 'ycg': 0.210, 'ybc': 0.050, 'osc_mount': 0.600, 'ocg': 0.050, 'obc': 0.001, 'other': 0.038},
        'ocg' : {'ymount': 0.400, 'ysc': 0.050, 'ycg': 0.010, 'ybc': 0.010, 'osc_mount': 0.001, 'ocg': 0.400, 'obc': 0.001, 'other': 0.028},
        'obc' : {'ymount': 0.010, 'ysc': 0.050, 'ycg': 0.001, 'ybc': 0.110, 'osc_mount': 0.050, 'ocg': 0.001, 'obc': 0.700, 'other': 0.078},
        'oth' : {'ymount': 0.050, 'ysc': 0.010, 'ycg': 0.050, 'ybc': 0.078, 'osc_mount': 0.001, 'ocg': 0.010, 'obc': 0.001, 'other': 0.800}
     }
"""

emissionprob = np.array([
                        [0.500, 0.050, 0.010, 0.050, 0.001, 0.350, 0.001, 0.038], 
                        [0.100, 0.800, 0.010, 0.010, 0.001, 0.050, 0.001, 0.028], 
                        [0.010, 0.050, 0.350, 0.010, 0.500, 0.001, 0.001, 0.078], 
                        [0.050, 0.010, 0.050, 0.700, 0.001, 0.010, 0.101, 0.078],
                        [0.001, 0.050, 0.210, 0.050, 0.600, 0.050, 0.001, 0.038],
                        [0.400, 0.050, 0.010, 0.010, 0.001, 0.400, 0.001, 0.028],
                        [0.010, 0.050, 0.001, 0.110, 0.050, 0.001, 0.700, 0.078],
                        [0.050, 0.010, 0.050, 0.078, 0.001, 0.010, 0.001, 0.800],
                        ])


In [114]:
# Hidden Markov Model with multinomial (discrete) emissions
model = hmm.MultinomialHMM(n_components=n_components,
                           n_iter=10,
                           verbose=False)

model.startprob_ = startprob
model.transmat_ = transmat
model.emissionprob_ = emissionprob
# model.n_features = 8

In [115]:
# observations = np.array([1, 1, 2, 2, 1, 0, 1, 2, 2, 0])
observations = np.array(pre_smooth)
observations2 = np.array(pre_smooth2)
#observations3 = np.array(pre_smooth3)
#observations4 = np.array(pre_smooth4)
#observations5 = np.array(pre_smooth5)
a,b = model.sample(5)
print a,b
print '=========='

n_samples = len(observations)
data = observations.reshape((n_samples, -1))
print data

n_samples2 = len(observations2)
data2 = observations2.reshape((n_samples2, -1))

#n_samples3 = len(observations3)
#data3 = observations3.reshape((n_samples3, -1))

#n_samples4 = len(observations4)
#data4 = observations4.reshape((n_samples4, -1))

#n_samples5 = len(observations5)
#data5 = observations5.reshape((n_samples5, -1))

[[2]
 [2]
 [4]
 [4]
 [3]] [4 4 4 4 4]
[[7]
 [7]
 [7]
 [0]
 [0]
 [0]
 [5]
 [5]
 [0]
 [5]
 [0]
 [5]
 [5]
 [0]
 [5]
 [7]
 [5]
 [0]
 [6]
 [4]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [4]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [4]
 [4]
 [4]
 [4]
 [4]
 [4]]


In [116]:
# decode(X, lengths=None, algorithm=None)[source]
# Find most likely state sequence corresponding to X.
# Will work best for organic tests

"""correct sequence
your_mount' if v == 0 
else 'your_side_control' if v == 1
else 'your_closed_guard' if v == 2
else 'your_back_control' if v == 3
else 'opponent_mount_or_sc' if v == 4
else 'opponent_closed_guard' if v == 5
else 'opponent_back_control' if v == 6
else 'OTHER' if v == 7


[3, 0, 1, 2, 5, 4, 6]

"""

"correct sequence\nyour_mount' if v == 0 \nelse 'your_side_control' if v == 1\nelse 'your_closed_guard' if v == 2\nelse 'your_back_control' if v == 3\nelse 'opponent_mount_or_sc' if v == 4\nelse 'opponent_closed_guard' if v == 5\nelse 'opponent_back_control' if v == 6\nelse 'OTHER' if v == 7\n\n\n[3, 0, 1, 2, 5, 4, 6]\n\n"

In [117]:
print 'TEST 1'

result = model.decode(data, algorithm='viterbi')
print 'pre smooth: {}'.format(pre_smooth)
print 'result accuracy {}%'.format(result[0])
print 'final result: {}'.format(result[1])

result_words = convert_to_words(result[1])
print '====================='
print 'pre smooth words: {}'.format(pre_smooth_words)
print '====================='
print 'result words: {}'.format(result_words)

print '\n'
print "pre smooth stats"
print get_position_stats(pre_smooth_words)

print '\n'

print 'result stats'
print get_position_stats(result_words)

print '******************'




TEST 1
pre smooth: [7 7 7 0 0 0 5 5 0 5 0 5 5 0 5 7 5 0 6 4 2 2 2 2 2 2 2 2 4 2 2 2 2 2 2 4 4
 4 4 4 4]
result accuracy -56.3499979936%
final result: [7 7 7 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 3 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2]
pre smooth words: ['OTHER', 'OTHER', 'OTHER', 'your_mount', 'your_mount', 'your_mount', 'opponent_closed_guard', 'opponent_closed_guard', 'your_mount', 'opponent_closed_guard', 'your_mount', 'opponent_closed_guard', 'opponent_closed_guard', 'your_mount', 'opponent_closed_guard', 'OTHER', 'opponent_closed_guard', 'your_mount', 'opponent_back_control', 'opponent_mount_or_sc', 'your_closed_guard', 'your_closed_guard', 'your_closed_guard', 'your_closed_guard', 'your_closed_guard', 'your_closed_guard', 'your_closed_guard', 'your_closed_guard', 'opponent_mount_or_sc', 'your_closed_guard', 'your_closed_guard', 'your_closed_guard', 'your_closed_guard', 'your_closed_guard', 'your_closed_guard', 'opponent_mount_or_sc', 'opponent_mount_or_sc', 'opponent_mount_or_sc'

In [118]:
print 'TEST2'
result2 = model.decode(data2, algorithm='viterbi')
print 'pre smooth: {}'.format(pre_smooth2)
print 'result accuracy {}%'.format(result2[0])
print 'final result: {}'.format(result2[1])

result_words2 = convert_to_words(result2[1])
print '====================='
print 'pre smooth words: {}'.format(pre_smooth_words2)
print '====================='
print 'result words: {}'.format(result_words2)

print '\n'
print "pre smooth stats"
print get_position_stats(pre_smooth_words2)

print '\n'

print 'result stats'
print get_position_stats(result_words2)

print '******************'

TEST2
pre smooth: [7 7 3 6 3 3 3 3 3 0 0 0 0 5 7 0 0 1 1 0 1 6 1 1 1 1 0 7 4 4 4 4 2 3 4 4 2
 2 2 2 2 4 2 5 0 0 5 5 5 5 0 5 5 5 2 4 4 4 4 4 4 4 4 4 4 4 4 4 6 6 4 6 6 6
 6 6 6 6 6 1 0]
result accuracy -120.080821923%
final result: [7 7 3 3 3 3 3 3 3 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 7 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 0 0 0 0 0 0 0 0 0 0 0 2 4 4 4 4 4 4 4 4 4 4 4 4 4 6 6 6 6 6 6
 6 6 6 6 6 1 1]
pre smooth words: ['OTHER', 'OTHER', 'your_back_control', 'opponent_back_control', 'your_back_control', 'your_back_control', 'your_back_control', 'your_back_control', 'your_back_control', 'your_mount', 'your_mount', 'your_mount', 'your_mount', 'opponent_closed_guard', 'OTHER', 'your_mount', 'your_mount', 'your_side_control', 'your_side_control', 'your_mount', 'your_side_control', 'opponent_back_control', 'your_side_control', 'your_side_control', 'your_side_control', 'your_side_control', 'your_mount', 'OTHER', 'opponent_mount_or_sc', 'opponent_mount_or_sc', 'opponent_mount_or_sc', 'opponent_mount_o

In [21]:
"""
print 'TEST3'
result3 = model.decode(data3, algorithm='viterbi')
print 'pre smooth: {}'.format(pre_smooth3)
print 'result accuracy {}%'.format(result3[0])
print 'final result: {}'.format(result3[1])

result_words3 = convert_to_words(result3[1])
print '====================='
print 'pre smooth words: {}'.format(pre_smooth_words3)
print '====================='
print 'result words: {}'.format(result_words3)

print '\n'
print "pre smooth stats"
print get_position_stats(pre_smooth_words3)

print '\n'

print 'result stats'
print get_position_stats(result_words3)

print '******************'
"""

TEST3
pre smooth: [7 7 6 1 6 6 1 6 6 6 3 3 3 1 0 5 0 0 0 0 1 1 1 1 1 1 1 0 4 2 2 2 2 2 2 2 2
 2 2 2 7 0 5 5 0 5 5 5 5 5 5 5 5 7 4 4 4 4 2 4 2 4 4 4 2 4 4 4 4 2 4 3 6 2
 4 6 3 4 6 6 6 6 6 6 6 6 6 7 5 7]
result accuracy -142.448382567%
final result: [7 7 6 6 6 6 6 6 6 6 3 3 3 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2
 2 2 2 7 0 0 0 0 0 0 0 0 0 0 0 0 7 2 2 2 2 2 2 2 4 4 4 4 4 4 4 4 4 4 6 6 4
 4 6 6 6 6 6 6 6 6 6 6 6 6 7 7 7]
pre smooth words: ['OTHER', 'OTHER', 'opponent_back_control', 'your_side_control', 'opponent_back_control', 'opponent_back_control', 'your_side_control', 'opponent_back_control', 'opponent_back_control', 'opponent_back_control', 'your_back_control', 'your_back_control', 'your_back_control', 'your_side_control', 'your_mount', 'opponent_closed_guard', 'your_mount', 'your_mount', 'your_mount', 'your_mount', 'your_side_control', 'your_side_control', 'your_side_control', 'your_side_control', 'your_side_control', 'your_side_control', 'your_side_control', 'your_mount',

In [22]:
"""
print 'TEST4'
result4 = model.decode(data4, algorithm='viterbi')
print 'pre smooth: {}'.format(pre_smooth4)
print 'result accuracy {}%'.format(result4[0])
print 'final result: {}'.format(result4[1])

result_words4 = convert_to_words(result4[1])
print '====================='
print 'pre smooth words: {}'.format(pre_smooth_words4)
print '====================='
print 'result words: {}'.format(result_words4)

print '\n'
print "pre smooth stats"
print get_position_stats(pre_smooth_words4)

print '\n'

print 'result stats'
print get_position_stats(result_words4)

print '******************'
"""

TEST4
pre smooth: [7 7 7 6 3 3 6 6 6 3 3 3 5 0 0 5 5 5 5 5 5 7 7 7 7 5 7 5 0 1 1 1 1 1 1 1 1
 1 1 1 5 7 2 2 2 2 7 7 7 2 2 2 7 7 7 2 2 7 7 7 7 2 2 2 5 5 5 5 5 5 5 5 5 5
 5 5 5 5 5 0 5 7 4 4 4 4 4 7 7 7 7 7 7 7 7 7 2 4 4 4 4 4 4 4 4 7 4 7 7 7 7
 4 4 2 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 0 0 7 5]
result accuracy -190.670795411%
final result: [7 7 7 3 3 3 3 3 3 3 3 3 0 0 0 0 0 0 0 0 0 7 7 7 7 0 0 0 0 1 1 1 1 1 1 1 1
 1 1 1 0 7 2 2 2 2 7 7 7 2 2 2 7 7 7 7 7 7 7 7 7 2 2 2 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 7 2 2 2 2 2 7 7 7 7 7 7 7 7 7 2 2 2 2 2 2 2 2 2 2 2 7 7 7 7
 2 4 4 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 0 0 0 0]
pre smooth words: ['OTHER', 'OTHER', 'OTHER', 'opponent_back_control', 'your_back_control', 'your_back_control', 'opponent_back_control', 'opponent_back_control', 'opponent_back_control', 'your_back_control', 'your_back_control', 'your_back_control', 'opponent_closed_guard', 'your_mount', 'your_mount', 'opponent_closed_guard', 'opponent_closed_guard', 'opponent_closed_guard', 'oppone

In [23]:
"""
print 'TEST5'
result5 = model.decode(data5, algorithm='viterbi')
print 'pre smooth: {}'.format(pre_smooth5)
print 'result accuracy {}%'.format(result5[0])
print 'final result: {}'.format(result5[1])

result_words5 = convert_to_words(result5[1])
print '====================='
print 'pre smooth words: {}'.format(pre_smooth_words5)
print '====================='
print 'result words: {}'.format(result_words5)

print '\n'
print "pre smooth stats"
print get_position_stats(pre_smooth_words5)

print '\n'

print 'result stats'
print get_position_stats(result_words5)
"""

TEST5
pre smooth: [7 7 2 3 3 3 3 3 3 3 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 0 4 4 4 4 4 4 2 2 4 4
 1 0 0 0 3 0 0 0 5 0 4 4 4 4 4 4 4 4 4 4 4 4 2 4 4 4 6 3 6 6 6 6 6 6 6 6 6
 1 1 5 7]
result accuracy -101.599747597%
final result: [7 7 3 3 3 3 3 3 3 3 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2
 1 0 0 0 0 0 0 0 0 0 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 6 6 6 6 6 6 6 6 6 6 6
 1 1 0 0]
pre smooth words: ['OTHER', 'OTHER', 'your_closed_guard', 'your_back_control', 'your_back_control', 'your_back_control', 'your_back_control', 'your_back_control', 'your_back_control', 'your_back_control', 'your_mount', 'your_mount', 'your_mount', 'your_mount', 'your_mount', 'your_mount', 'your_mount', 'your_mount', 'your_mount', 'your_side_control', 'your_side_control', 'your_side_control', 'your_side_control', 'your_side_control', 'your_side_control', 'your_side_control', 'your_mount', 'opponent_mount_or_sc', 'opponent_mount_or_sc', 'opponent_mount_or_sc', 'opponent_mount_or_sc', 'opponent_mount_or_sc', 'opp